In [1]:
!pip install vllm==0.6.0
!pip install scikit-learn
!pip install fitz
!pip install praw
!pip install yfinance
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 MB 17.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 23.0 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 MB 19.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 25.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 54.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 147.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 117.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 87.7 MB/

In [2]:
import pandas as pd
import fitz
import io
import time
import json
import os
import urllib.request
import praw
from datetime import datetime, timedelta
import yfinance as yf
import requests
from transformers import AutoTokenizer
import json
import re

In [3]:

import string
from vllm import LLM, SamplingParams
# LLM
model_id='irene93/functioncall_stkissue'
llm = LLM(model=model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

INFO 03-17 04:20:02 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='irene93/functioncall_stkissue', speculative_config=None, tokenizer='irene93/functioncall_stkissue', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=irene93/functioncall_stkissue, use_v2_block_manager=False, num_scheduler_steps=1, enable_prefix_caching=False, use

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 03-17 04:20:06 model_runner.py:926] Loading model weights took 14.2487 GB
INFO 03-17 04:20:08 gpu_executor.py:122] # GPU blocks: 61092, # CPU blocks: 4681
INFO 03-17 04:20:13 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-17 04:20:13 model_runner.py:1221] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 03-17 04:20:26 model_runner.py:1335] Graph capturing finished in 13 secs.


In [4]:
system_prompt = """당신은 다양한 기능을 호출할 수 있는 AI 모델입니다. 사용자의 요청에 따라 특정 함수를 호출하고, 함수의 시그니처는 <tools></tools> XML 태그 내에 제공됩니다. 함수를 호출할 때는 함수에 필요한 정확한 값을 추정하지 말고 사용자가 제공한 값에 따라 함수를 실행해야 합니다.

아래는 사용 가능한 함수들과 각각의 파라미터에 대한 설명입니다:

get_issues_summarized

설명: 특정 회사 또는 키워드에 대한 이슈를 검색하고 요약합니다.
파라미터:
keyword: 이슈/현황을 검색하고자 하는 회사명 또는 키워드.
days: 검색하고자 하는 기간(일 단위 (integer)).

get_reddit_hotissue

설명: 금융시장에서 핫한 이슈를 요약합니다.
파라미터:
days: 검색하고자 하는 기간(일 단위).

{name: get_earnings,
설명: 기업의 재무재표 또는 현금흐름을 가져오고 분석합니다. 성장률과 같이 이전 년도 데이터가 필요한경우, 이전 년도 데이터도 한번 더 호출하세요
파라미터:{symbol: 실적 데이터를 찾고자 하는 기업의 심볼.
analysis_type: 분석 유형(growth, profitability, stability, valuation, cashflow, dividend, cost, NA).
type_: 데이터 타입(yearly 또는 quarter).
year: 데이터를 찾고자 하는 연도 (명시하지 않을경우, 데이터가 존재하는 최근 연도의 데이터를 참조합니다.).
quarter: 데이터를 찾고자 하는 분기(명시하지않은 경우 최근 데이터를 조회하도록 -1 을 입력합니다).}}

get_consensus

설명: 기업의 EPS 컨센서스 데이터 또는 매수/매도/홀드 의견을 가져와서 분석합니다.
파라미터:
symbol: 데이터를 찾고자 하는 기업의 심볼.
year: 데이터를 찾고자 하는 연도.
quarter: 데이터를 찾고자 하는 분기.
각 함수 호출 시, JSON 객체를 사용하여 함수 이름과 인자들을 <tool_call></tool_call> XML 태그 내에 명시해야 합니다. 함수 호출 예시는 다음과 같습니다:

xml
Copy
<tool_call>
{
    "name": "get_earnings",
    "arguments": {
        "symbol": "AAPL",
        "analysis_type": "growth",
        "type_": "yearly" ,
        "year": "2024",
        "quarter": "-1"
    }
}
</tool_call>
각 함수의 인자 값을 정확하게 지정해 주세요. 특히, 연도와 분기를 설정할 때 현재 날짜가 1월이나 2월인 경우, 최근 연도의 데이터를 참조하도록 주의해야 합니다.
**주의사항
함수를 호출할때를 제외하고 한국어로 대답하세요.

"""

In [6]:
Q1ws2e3=][
]tyhgbvfrc dexsz
]

In [75]:
import string

def get_response(messages):
    # 기본 시스템 메시지 추가
    template = string.Template("<|im_start|>system\n${system_content}<|im_end|>")
    
    prompts = template.safe_substitute({"system_content": messages[0]["content"]})
    
    # 유저 및 어시스턴트 메시지를 반복적으로 추가
    
    for msg in messages[1:]:
        if msg['role'] == 'user':
            prompts += f"<|im_start|>user\n{msg['content']}<|im_end|>"
        elif msg['role'] == 'assistant':
            prompts += f"<|im_start|>assistnat{msg['content']}<|im_end|>"
    
    
    # 최종 출력 확인"
    # print(prompt)
    
    # 샘플링 파라미터 설정
    sampling_params = SamplingParams(
        temperature=0,
        max_tokens=1024,
        stop=["<|im_end|>"],  # 문자열로 중지 토큰 지정
        repetition_penalty=1.0
    )
    
    # 생성
    outputs = llm.generate([prompts], sampling_params)
    
    # 결과 출력
    for output in outputs:
        print()
        return output.outputs[0].text
        

In [76]:
system_prompt_date = '오늘 날짜는 2025-03-15 입니다.' + system_prompt
user_prompt = "테슬라(Tesla)의 최근 주식 매수/매도 분기 전망은 어떻게 되나요?"

messages= [
    {"role": "system", "content": system_prompt_date},
    {"role": "user", "content": user_prompt},
    {"role": "assistant", "content": ""}  # AI의 응답이 들어갈 자리
]

In [77]:
get_response(messages)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.28it/s, est. speed input: 1624.18 toks/s, output: 75.38 toks/s]

'{"name":"get_consensus","arguments": {"symbol": "TSLA", "year": "2024", "quarter": "-1"}}'

In [56]:
import pickle
with open('./data/test_trl_messages.pkl','rb') as f:
    test_messages = pickle.load(f)

In [95]:
query_qwen_model(system_prompt, user_prompt)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.02it/s, est. speed input: 1388.03 toks/s, output: 76.77 toks/s]


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.91s/it, est. speed input: 302.10 toks/s, output: 87.05 toks/s]

'assistnat\n테슬라(TSLA)의 최근 주식 매수/매도 분기 전망을 보면 다음과 같습니다:\n\n- **EPS 실제 결과 vs. 추정치**: 테슬라는 2025년 1월 29일 발표한 실제 EPS(주당순이익)가 0.73으로, 추정치 0.774보다 약간 낮은 결과를 보였습니다. EPS에서 약 5.68%의 마이너스 서프라이즈가 있었습니다.\n\n- **주가 목표 가격**: 현재 주가는 249.98달러이며, 최고 목표가는 550.0달러, 최저 목표가는 120.0달러입니다. 평균 목표가는 343.11달러, 중간값은 376.5달러입니다.\n\n이 정보를 바탕으로 투자 결정을 고려하실 수 있습니다. 투자 관련 전문가의 조언을 고려하거나 본인의 투자 전략을 확인하는 것이 좋습니다.'

In [94]:
def call_function(name, args):
    if name == "get_reddit_issues_summarized":
        return get_reddit_issues_summarized(**args)
    if name == "get_reddit_hotissue":
        return get_reddit_hotissue(**args)
    if name == "get_consensus":
        return get_consensus(**args)
    if name == "get_earnings":
        return get_earnings(**args)


def query_qwen_model(system_prompt, question):
    full_messages = [{"role": "system", "content": system_prompt}]
    messages = full_messages
    messages.append({"role": "user", "content": question})
    response_message = get_response(messages)
    
    if '<tool_call>' in response_message:
        tool_call_message= {'role':'assistant','content':response_message}
        messages.append(tool_call_message)
        toolcall_pattern = r'<tool_call>(.*?)</tool_call>'
        tool_call_json = re.findall(toolcall_pattern, response_message, re.DOTALL)
        for tool_call_str in tool_call_json:

            tool_call =  json.loads(tool_call_str)

            function_name = tool_call['name']
            function_args = tool_call['arguments']

            function_response = call_function(function_name, function_args)
            tool_response = {'role':'user','content':'<tool_reponse>'+function_response+'</tool_reponse>'}
            messages.append(tool_response)
    final_message = get_response(messages)
    return final_message



In [87]:
full_messages = [{"role": "system", "content": system_prompt}]
messages = full_messages
messages.append({"role": "user", "content": user_prompt})
response_message = get_response(messages)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.02it/s, est. speed input: 1389.95 toks/s, output: 76.88 toks/s]

In [90]:
toolcall_pattern = r'<tool_call>(.*?)</tool_call>'
tool_call_json = re.findall(toolcall_pattern, response_message, re.DOTALL)

In [93]:
tool_call_str = tool_call_json[0]
json.loads(tool_call_str)

{'name': 'get_consensus',
 'arguments': {'symbol': 'TSLA', 'year': '2024', 'quarter': '-1'}}

In [1]:
stakc =[0,1]

In [2]:
stakc.pop()

1